In [1]:
def cnn_mlp(train_x1, train_x2, train_y):
    # cnn和mlp网络进行组合
    # x1是40000长度的内容特征,x2是2500长度的文件特征
    tf.keras.backend.clear_session()
    # train_x1=np.reshape(train_x1, (-1, 200, 200))
    input1 = layers.Input(shape=(200, 200))
    input2 = layers.Input(shape=2500)
    x1 = layers.Conv1D(16, kernel_size=5, name="conv_1", activation="relu")(input1)
    x1 = layers.MaxPool1D(name="maxpool1")(x1)
    x1 = layers.Conv1D(128, kernel_size=2, name="conv_2", activation="relu")(x1)
    x1 = layers.MaxPool1D(name="maxpool2")(x1)
    x1 = layers.Flatten()(x1)
    x2 = layers.Dense(1200, activation='tanh')(input2)
    c = layers.concatenate([x1, x2], axis=1)
    c = layers.Dense(600, activation='sigmoid')(c)
    c = layers.Dense(300, activation='sigmoid')(c)
    c = layers.Dense(150, activation='sigmoid')(c)
    c = layers.Dense(60, activation='sigmoid')(c)
    outputs = layers.Dense(10, activation='softmax')(c)
    model = models.Model(inputs=[input1, input2], outputs=outputs)
    model.summary()

    import datetime
    import os

    stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    # 在 Python3 下建议使用 pathlib 修正各操作系统的路径
    from pathlib import Path
    stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    logdir = str(Path('./data/autograph/' + stamp))

    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=["accuracy", "Recall", "Precision", "AUC"]

    )

    history = model.fit([train_x1, train_x2], train_y, epochs=30, validation_split=0.2,
                        callbacks=[tensorboard_callback], workers=4, shuffle=True)

    return model


In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

dataset = "fmnist"
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 归一化
X_train = X_train / 255.
X_test = X_test / 255.